In [2]:
# clonging the repo for dataset
!git clone 'https://github.com/muxspace/facial_expressions'

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 22.17 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Updating files: 100% (13996/13996), done.


In [6]:
# mapping emotions and images from legend.csv file
import csv
data = {}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader = csv.reader(f)
  next(reader)
  for row in reader:
    key = row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key] = [row[1]]


In [7]:
emotion_list = list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [10]:
# splitting data to training and testing
import os
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [11]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training/',emotion))
  os.mkdir(os.path.join('master_data/testing/',emotion))

In [12]:
from shutil import copyfile
split_size = 0.8

for emotion, images in data.items():
  train_size = int(split_size*len(images))
  train_images = images[:train_size]
  test_images = images[train_size:]

  for image in train_images:
    src = os.path.join('/content/facial_expressions/images', image)
    des = os.path.join('master_data/training/',emotion, image)
    copyfile(src,des)
  for image in test_images:
    src = os.path.join('/content/facial_expressions/images', image)
    des = os.path.join('master_data/testing/',emotion, image)
    copyfile(src,des)

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [18]:
from tensorflow.keras.losses import categorical_crossentropy
# building and compiling model using CNN
model = tf.keras.models.Sequential(
    [
        Conv2D(16, (3,3), activation = 'relu', input_shape = (100,100,3)),
        MaxPooling2D(2,2),
        Conv2D(32, (3,3), activation = 'relu'),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation = 'relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(512, activation = 'relu'),
        Dense(8, activation = 'softmax')
    ]
)
model.compile(optimizer = Adam(lr = 0.01), loss = categorical_crossentropy, metrics = ['acc'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 49, 49, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 23, 23, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 10, 10, 64)       

In [20]:
train_dir = '/content/master_data/training'
test_dir = '/content/master_data/testing'

train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size = (100,100), class_mode = 'categorical',
    batch_size = 128
)

test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_generator = test_datagen.flow_from_directory(
    test_dir, target_size = (100,100), class_mode = 'categorical',
    batch_size = 128
)

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [21]:
# to prevent the model from unnecessary training
es = EarlyStopping(monitor = 'val_acc', patience = 2, min_delta = 0.01)

In [23]:
model.fit_generator(train_generator, epochs = 10, verbose = 1, validation_data = test_generator, callbacks = [es])

<ipython-input-23-3ea39573b049>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs = 10, verbose = 1, validation_data = test_generator, callbacks = [es])


Epoch 1/10
86/86 [==============================] - 138s 2s/step - loss: 0.9573 - acc: 0.5998 - val_loss: 1.3477 - val_acc: 0.6327
Epoch 2/10
86/86 [==============================] - 120s 1s/step - loss: 0.5905 - acc: 0.7984 - val_loss: 1.5330 - val_acc: 0.6681
Epoch 3/10
86/86 [==============================] - 120s 1s/step - loss: 0.4919 - acc: 0.8283 - val_loss: 1.6240 - val_acc: 0.6853
Epoch 4/10
86/86 [==============================] - 130s 1s/step - loss: 0.4400 - acc: 0.8486 - val_loss: 1.6374 - val_acc: 0.6882
Epoch 5/10
86/86 [==============================] - 126s 1s/step - loss: 0.3866 - acc: 0.8649 - val_loss: 1.8187 - val_acc: 0.6842
